<span style="font-family:Lucida Bright;">
<p style="margin-bottom:0.5cm"></p>
<center>
<font size="8"><b>Deep Learning, Fall 2021</b></font>
<p style="margin-bottom:0.2cm"></p>
<font size="3"><b>Final Project:</b></font>
<p style="margin-bottom:0.1cm"></p>
<font size="5"><b>Enhancing Voices for Better Speech Intelligibility</b></font>
<p style="margin-bottom:2cm"></p>
<font size="6"><b>Title</b></font>
</center>
<p style="margin-bottom:2cm"></p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inizializazione" data-toc-modified-id="Inizializazione-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inizializazione</a></span></li><li><span><a href="#Prognosi" data-toc-modified-id="Prognosi-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prognosi</a></span></li></ul></div>

# Inizializazione

The initialization procedure is defined in the notebook: [Initializzation](intialization.ipynb).

In [1]:
%run ./initialization.ipynb

# Prognosi

ModuleNotFoundError: No module named 'matlab'